## INTRUSION ZONE DETECTION AND IMAGE SAVING

### FINDING COORDINATES OF ZONE

In [15]:
import cv2

points = []


def select_zone(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))
        print(f"Point selected: {x}, {y}")
        cv2.circle(img, (x, y), 5, (0, 255, 0), -1)
        cv2.imshow("Define Zone", img)

# Load image
img = cv2.imread('500_1.jpg')
cv2.imshow("Define Zone", img)
cv2.setMouseCallback("Define Zone", select_zone)

print("\n Click to select the corners of the zone on the image.")
print("Press 'q' when done.\n")

while True:
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cv2.destroyAllWindows()

# Print final coordinates
print("\n🟢 Final zone coordinates:")
print(points)



 Click to select the corners of the zone on the image.
Press 'q' when done.

Point selected: 178, 109
Point selected: 522, 117
Point selected: 438, 362
Point selected: 139, 354

🟢 Final zone coordinates:
[(178, 109), (522, 117), (438, 362), (139, 354)]


### INTRUSION ZONE DETECTION AND IMAGE SAVING

In [3]:
import cv2
import os
from ultralytics import YOLO
import numpy as np


zone_coords = [(175, 85), (141, 310), (442, 325), (443, 97)]  
input_folder = r'C:\Users\ARJUN\Desktop\CV PROJECT\data\tracked_trimmed_object'
output_folder = r'C:\Users\ARJUN\Desktop\CV PROJECT\data\intrusion'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Load YOLOv8 model
model = YOLO('yolov8n.pt')  

# Function to check if center of object is inside polygon
def point_in_polygon(x, y, polygon):
    return cv2.pointPolygonTest(np.array(polygon, np.int32), (int(x), int(y)), False) >= 0

# Loop over all images
for img_name in sorted(os.listdir(input_folder)):
    if not img_name.lower().endswith(('.jpg', '.png', '.jpeg')):
        continue
    
    img_path = os.path.join(input_folder, img_name)
    frame = cv2.imread(img_path)
    if frame is None:
        continue

    results = model(frame)[0]
    intrusion = False

    for box in results.boxes:
        cls_id = int(box.cls[0])
        if results.names[cls_id] == 'person':
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            if point_in_polygon(cx, cy, zone_coords):
                intrusion = True
                # Draw red bbox
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
            else:
                # Draw green bbox
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Draw the polygon zone
    cv2.polylines(frame, [np.array(zone_coords, np.int32)], isClosed=True, color=(255, 0, 0), thickness=2)
    
    if intrusion:
        cv2.imwrite(os.path.join(output_folder, img_name), frame)
        print(f"[ALERT] Intrusion detected in {img_name}")


C:\Users\ARJUN\anaconda3\envs\myenv\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(



0: 448x640 1 car, 1 fire hydrant, 176.9ms
Speed: 6.4ms preprocess, 176.9ms inference, 1.9ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 2 persons, 1 car, 117.5ms
Speed: 4.1ms preprocess, 117.5ms inference, 1.2ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 person, 2 cars, 116.4ms
Speed: 3.2ms preprocess, 116.4ms inference, 1.4ms postprocess per image at shape (1, 3, 448, 640)
[ALERT] Intrusion detected in 500_100.jpg

0: 448x640 1 person, 3 cars, 134.5ms
Speed: 3.9ms preprocess, 134.5ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 640)
[ALERT] Intrusion detected in 500_101.jpg

0: 448x640 1 person, 3 cars, 104.0ms
Speed: 2.7ms preprocess, 104.0ms inference, 1.7ms postprocess per image at shape (1, 3, 448, 640)
[ALERT] Intrusion detected in 500_102.jpg

0: 448x640 1 person, 3 cars, 90.9ms
Speed: 2.6ms preprocess, 90.9ms inference, 1.2ms postprocess per image at shape (1, 3, 448, 640)
[ALERT] Intrusion detected in 500_103.jpg

0: 448x640 1

### SMS Alert

In [7]:
from twilio.rest import Client

account_sid = 'AC1ef1413d802ac2993273c3d366c4afdb'
auth_token = '140fa15f96af47c6ec3b107cbfbb27ff'
client = Client(account_sid, auth_token)

message = client.messages.create(
body='🚨 Intrusion detected in restricted zone!',
from_='+19124937085', 
to='+918714554675'  
)

print(f"SMS sent. SID: {message.sid}")


SMS sent. SID: SMed3c8388bf8b15ecc087acf24596dd0c


In [9]:
from twilio.rest import Client

# --- SMS Setup (Twilio) ---
account_sid = 'AC1ef1413d802ac2993273c3d366c4afdb'
auth_token = '140fa15f96af47c6ec3b107cbfbb27ff'
from_='+19124937085',  # Your Twilio number
to='+918714554675'# Your phone number in India

client = Client(account_sid, auth_token)


sms_sent = False

import cv2
import os
from ultralytics import YOLO
import numpy as np

# --- SETTINGS ---
zone_coords = [(175, 85), (141, 310), (442, 325), (443, 97)]  # Polygon for restricted area
input_folder = r'C:\Users\ARJUN\Desktop\CV PROJECT\tracked_new'
output_folder = r'C:\Users\ARJUN\Desktop\CV PROJECT\intrusion'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Load YOLOv8 model
model = YOLO('yolov8n.pt')  # Or yolov8s.pt for more accuracy

# Function to check if center of object is inside polygon
def point_in_polygon(x, y, polygon):
    return cv2.pointPolygonTest(np.array(polygon, np.int32), (int(x), int(y)), False) >= 0

# Loop over all images
for img_name in sorted(os.listdir(input_folder)):
    if not img_name.lower().endswith(('.jpg', '.png', '.jpeg')):
        continue
    
    img_path = os.path.join(input_folder, img_name)
    frame = cv2.imread(img_path)
    if frame is None:
        continue

    results = model(frame)[0]
    intrusion = False

    for box in results.boxes:
        cls_id = int(box.cls[0])
        if results.names[cls_id] == 'person':
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            if point_in_polygon(cx, cy, zone_coords):
                intrusion = True
                # Draw red bbox
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
            else:
                # Draw green bbox
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Draw the polygon zone
    cv2.polylines(frame, [np.array(zone_coords, np.int32)], isClosed=True, color=(255, 0, 0), thickness=2)
    
    if intrusion:
        cv2.imwrite(os.path.join(output_folder, img_name), frame)
        print(f"[ALERT] Intrusion detected in {img_name}")
        
        if not sms_sent:
            message = client.messages.create(
                body="⚠️ Intrusion detected in restricted area!",
                from_=from_,
                to=to
            )
            print("[SMS SENT]")
            sms_sent = True





0: 448x640 1 car, 1 fire hydrant, 293.0ms
Speed: 7.5ms preprocess, 293.0ms inference, 2.8ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 2 persons, 1 car, 197.2ms
Speed: 4.3ms preprocess, 197.2ms inference, 3.2ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 person, 2 cars, 185.2ms
Speed: 4.6ms preprocess, 185.2ms inference, 2.8ms postprocess per image at shape (1, 3, 448, 640)
[ALERT] Intrusion detected in 500_100.jpg
[SMS SENT]

0: 448x640 1 person, 3 cars, 220.5ms
Speed: 6.0ms preprocess, 220.5ms inference, 2.4ms postprocess per image at shape (1, 3, 448, 640)
[ALERT] Intrusion detected in 500_101.jpg

0: 448x640 1 person, 3 cars, 244.1ms
Speed: 4.5ms preprocess, 244.1ms inference, 2.2ms postprocess per image at shape (1, 3, 448, 640)
[ALERT] Intrusion detected in 500_102.jpg

0: 448x640 1 person, 3 cars, 203.8ms
Speed: 7.8ms preprocess, 203.8ms inference, 2.7ms postprocess per image at shape (1, 3, 448, 640)
[ALERT] Intrusion detected in 500_103.jpg


In [45]:
import requests
requests.get('http://192.168.101.68/on')  # replace with your ESP IP


<Response [200]>

In [43]:
import requests
requests.get('http://192.168.101.68/off')

<Response [200]>

In [47]:
from twilio.rest import Client
import cv2
import os
from ultralytics import YOLO
import numpy as np
import requests  # New import for HTTP

# --- SMS Setup (Twilio) ---
account_sid = '*********************'
auth_token = '************************'
from_ = '************'  # Twilio number
to = '************'    # Your number

client = Client(account_sid, auth_token)
sms_sent = False
led_on = False

# --- SETTINGS ---
zone_coords = [(175, 85), (141, 310), (442, 325), (443, 97)]
input_folder = r'C:\Users\ARJUN\Desktop\CV PROJECT\tracked_trimmed_object'
output_folder = r'C:\Users\ARJUN\Desktop\CV PROJECT\intrusion'

# ESP8266 IP
esp_ip = "http://192.168.101.68"  # Replace with your NodeMCU IP

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Load YOLOv8 model
model = YOLO('yolov8n.pt')

def point_in_polygon(x, y, polygon):
    return cv2.pointPolygonTest(np.array(polygon, np.int32), (int(x), int(y)), False) >= 0

# --- Loop over images ---
for img_name in sorted(os.listdir(input_folder)):
    if not img_name.lower().endswith(('.jpg', '.png', '.jpeg')):
        continue
    
    img_path = os.path.join(input_folder, img_name)
    frame = cv2.imread(img_path)
    if frame is None:
        continue

    results = model(frame)[0]
    intrusion = False

    for box in results.boxes:
        cls_id = int(box.cls[0])
        if results.names[cls_id] == 'person':
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            if point_in_polygon(cx, cy, zone_coords):
                intrusion = True
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
            else:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    cv2.polylines(frame, [np.array(zone_coords, np.int32)], isClosed=True, color=(255, 0, 0), thickness=2)
    
    if intrusion:
        cv2.imwrite(os.path.join(output_folder, img_name), frame)
        print(f"[ALERT] Intrusion detected in {img_name}")

        if not sms_sent:
            try:
                message = client.messages.create(
                    body="⚠️ Intrusion detected in restricted area!",
                    from_=from_,
                    to=to
                )
                print("[SMS SENT]")
                sms_sent = True
            except Exception as e:
                print(f"[SMS ERROR] {e}")
        
        if not led_on:
            try:
                requests.get(f"{esp_ip}/on")
                print("[LED ON]")
                led_on = True
            except Exception as e:
                print(f"[LED ERROR] {e}")
    else:
        if led_on:
            try:
                requests.get(f"{esp_ip}/off")
                print("[LED OFF]")
                led_on = False
            except Exception as e:
                print(f"[LED ERROR] {e}")



0: 448x640 1 car, 1 fire hydrant, 85.7ms
Speed: 6.6ms preprocess, 85.7ms inference, 1.8ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 2 persons, 1 car, 79.2ms
Speed: 1.7ms preprocess, 79.2ms inference, 0.8ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 person, 2 cars, 124.9ms
Speed: 1.8ms preprocess, 124.9ms inference, 0.9ms postprocess per image at shape (1, 3, 448, 640)
[ALERT] Intrusion detected in 500_100.jpg
[SMS SENT]
[LED ON]

0: 448x640 1 person, 3 cars, 117.1ms
Speed: 6.3ms preprocess, 117.1ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)
[ALERT] Intrusion detected in 500_101.jpg

0: 448x640 1 person, 3 cars, 120.2ms
Speed: 1.6ms preprocess, 120.2ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)
[ALERT] Intrusion detected in 500_102.jpg

0: 448x640 1 person, 3 cars, 76.9ms
Speed: 2.4ms preprocess, 76.9ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)
[ALERT] Intrusion detected in 500_103.j

### Intrusion Integrated with IoT alert and SMS Alert

In [17]:
from twilio.rest import Client
import cv2
import os
from ultralytics import YOLO
import numpy as np
import time
import requests

# --- SMS Setup (Twilio) ---
account_sid = 'AC1ef1413d802ac2993273c3d366c4afdb'
auth_token = '140fa15f96af47c6ec3b107cbfbb27ff'
from_ = '+19124937085'
to = '+918714554675'
client = Client(account_sid, auth_token)

# --- ESP8266 Setup ---
esp_ip = "http://192.168.101.68"  # Replace with your NodeMCU IP

# --- Image & Detection Settings ---
zone_coords = [(175, 85), (141, 310), (442, 325), (443, 97)]  # Polygon zone
input_folder = r'C:\Users\ARJUN\Desktop\CV PROJECT\data\tracked_trimmed_object'
output_folder = r'C:\Users\ARJUN\Desktop\CV PROJECT\data\intrusion'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

model = YOLO('yolov8n.pt')  # Use yolov8s.pt for better accuracy

def point_in_polygon(x, y, polygon):
    return cv2.pointPolygonTest(np.array(polygon, np.int32), (int(x), int(y)), False) >= 0

sms_sent = False

# --- Main Loop ---
for img_name in sorted(os.listdir(input_folder)):
    if not img_name.lower().endswith(('.jpg', '.png', '.jpeg')):
        continue

    img_path = os.path.join(input_folder, img_name)
    frame = cv2.imread(img_path)
    if frame is None:
        continue

    results = model(frame)[0]
    intrusion = False

    for box in results.boxes:
        cls_id = int(box.cls[0])
        if results.names[cls_id] == 'person':
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

            if point_in_polygon(cx, cy, zone_coords):
                intrusion = True
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)

                # 🟡 Turn on LED for 1 second
                try:
                    requests.get(f"{esp_ip}/on")
                    print("[LED ON]")
                    time.sleep(1)
                    requests.get(f"{esp_ip}/off")
                    print("[LED OFF]")
                except Exception as e:
                    print(f"[LED ERROR] {e}")
            else:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Draw Zone
    cv2.polylines(frame, [np.array(zone_coords, np.int32)], isClosed=True, color=(255, 0, 0), thickness=2)

    if intrusion:
        cv2.imwrite(os.path.join(output_folder, img_name), frame)
        print(f"[ALERT] Intrusion detected in {img_name}")

        if not sms_sent:
            try:
                message = client.messages.create(
                    body="⚠️ Intrusion detected in restricted area!",
                    from_=from_,
                    to=to
                )
                print("[SMS SENT]")
                sms_sent = True
            except Exception as e:
                print(f"[SMS ERROR] {e}")



0: 448x640 1 car, 1 fire hydrant, 276.9ms
Speed: 22.9ms preprocess, 276.9ms inference, 8.3ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 2 persons, 1 car, 280.6ms
Speed: 4.0ms preprocess, 280.6ms inference, 2.2ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 person, 2 cars, 228.8ms
Speed: 4.4ms preprocess, 228.8ms inference, 2.6ms postprocess per image at shape (1, 3, 448, 640)
[LED ON]
[LED OFF]
[ALERT] Intrusion detected in 500_100.jpg
[SMS SENT]

0: 448x640 1 person, 3 cars, 375.9ms
Speed: 6.7ms preprocess, 375.9ms inference, 2.5ms postprocess per image at shape (1, 3, 448, 640)
[LED ON]
[LED OFF]
[ALERT] Intrusion detected in 500_101.jpg

0: 448x640 1 person, 3 cars, 232.3ms
Speed: 6.3ms preprocess, 232.3ms inference, 2.6ms postprocess per image at shape (1, 3, 448, 640)
[LED ON]
[LED OFF]
[ALERT] Intrusion detected in 500_102.jpg

0: 448x640 1 person, 3 cars, 283.7ms
Speed: 6.9ms preprocess, 283.7ms inference, 2.5ms postprocess per image at shape (

### Viewing

In [21]:
import cv2
import os
import re

folder = r'C:\Users\ARJUN\Desktop\CV PROJECT\Data\intrusion'
images = [img for img in os.listdir(folder) if img.endswith('.jpg')]
images.sort(key=lambda x: [int(t) if t.isdigit() else t for t in re.split('(\d+)', x)])

for image in images:
    img_path = os.path.join(folder, image)
    frame = cv2.imread(img_path)
    if frame is None:
        continue
    cv2.imshow("Fast Video", frame)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
C:\Users\ARJUN\AppData\Local\Temp\ipykernel_11328\1584740895.py:7: SyntaxWarning: invalid escape sequence '\d'
  images.sort(key=lambda x: [int(t) if t.isdigit() else t for t in re.split('(\d+)', x)])
